# K-Body Noise Reconstruction


Follows https://arxiv.org/pdf/2303.17714v1

In [1]:
%load_ext autoreload
%autoreload 2


In [286]:
import cirq

qubits = cirq.LineQubit.range(3)
process_circuit = cirq.Circuit(cirq.ZZ(*qubits[0:2]), cirq.X(qubits[2]))


class IndependentDepolariseNoiseModel(cirq.NoiseModel):
    """Applies single and two qubit depolarising channels independently"""

    def __init__(self, single_qubit_error: float, two_qubit_error: float) -> None:
        """Args:
        single_qubit_error: Single qubit pauli error
        two_qubit_error: Two qubit pauli error
        """
        super().__init__()
        self.single_qubit_error = single_qubit_error
        self.two_qubit_error = two_qubit_error

        self.single_qubit_depolarise = cirq.DepolarizingChannel(
            p=3 / 4 * single_qubit_error, n_qubits=1
        )
        self.two_qubit_depolarise = cirq.DepolarizingChannel(
            p=15 / 16 * two_qubit_error, n_qubits=2
        )

    def noisy_operation(self, operation: cirq.Operation) -> list[cirq.OP_TREE]:
        """Produces a list of operations by applying each noise model
        to the provided operation depending on the number of qubits it acts on.
        """
        if len(operation.qubits) == 1:
            return [operation, self.single_qubit_depolarise(*operation.qubits)]

        if len(operation.qubits) == 2:
            return [operation, self.two_qubit_depolarise(*operation.qubits)]

        return [operation]


noise = IndependentDepolariseNoiseModel(single_qubit_error=0.0001, two_qubit_error=0.04)

sim = cirq.DensityMatrixSimulator(
    noise=noise,
)

In [287]:
import supermarq

experiment = supermarq.qcvv.KNR(process_circuit, num_parallel_supports=1)
experiment.prepare_experiment(num_circuits=20, cycle_depths=[1, 3])
experiment.run_with_simulator(simulator=sim, repetitions=100)
if experiment.collect_data():
    result = experiment.analyze_results()

Building circuits:   0%|          | 0/720 [00:00<?, ?it/s]

Simulating circuits:   0%|          | 0/720 [00:00<?, ?it/s]

In [288]:
result

,pauli_channel,orbital_fidelity,marginal_probability
0,IIX,1.000250,0.000751
1,IIY,0.999500,-0.000751
2,IIZ,1.000000,0.000250
3,IXI,0.961929,0.032422
4,IYI,0.958909,0.054688
5,IZI,0.961416,0.040494
6,XII,0.957849,0.072628
7,XXI,0.957261,0.019860
8,XYI,0.967848,0.027861
9,XZI,0.962725,0.031590
